In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, CuDNNLSTM

(x_train, original_y_train), (x_test, original_y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
translation_dict = {
    '0': [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,],
    '1': [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,],
    '2': [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,],
    '3': [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,],
    '4': [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,],
    '5': [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,],
    '6': [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,],
    '7': [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,],
    '8': [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,],
    '9': [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,]
}

y_train = np.empty((original_y_train.shape[0], 10), dtype=np.float32)
i = 0
for number in original_y_train:
  y_train[i] = translation_dict[str(number)]
  i += 1

y_test = np.empty((original_y_test.shape[0], 10), dtype=np.float32)
i = 0
for number in original_y_test:
  y_test[i] = translation_dict[str(number)]
  i += 1

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
from keras.layers import CuDNNGRU, SimpleRNN  # LSTM, GRU

model = Sequential()
model.add(CuDNNGRU(128, input_shape=(x_train.shape[1:]), return_sequences=True))       # model.add(GRU(128, activation='relu', input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNGRU(128))                                                               # model.add(GRU(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# model = Sequential()
# model.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))    # model.add(LSTM(128, activation='relu', input_shape=(x_train.shape[1:]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(CuDNNLSTM(128))                                                            # model.add(LSTM(128, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation='softmax'))

# model = Sequential()
# model.add(SimpleRNN(128, input_shape=(x_train.shape[1:]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(SimpleRNN(128))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.show()

In [ ]:
model.save('model.h5')